# Problem 2 – Fractional Parts of Cube Roots  
**Author:** Michael Ferry  
**Date:** October 2025  

This problem focuses on recreating the constants listed on page 11 of the **Secure Hash Standard (FIPS PUB 180-4)**.  
These constants are the **first 32 bits of the fractional parts of the cube roots** of the first 64 prime numbers.

We’ll:
1. Write a function `primes(n)` to generate the first n primes.  
2. Use it to calculate cube roots for the first 64 primes.  
3. Extract the fractional part, scale it by 2³², and convert to 32-bit integers.  
4. Display the results in hexadecimal.  
5. Compare them to the official constants from FIPS 180-4 (Section 4.2.2).


In [2]:
import numpy as np
from math import pow, floor

print("Problem 2 imports loaded successfully.")


Problem 2 imports loaded successfully.


### Step 1 — Generate the first n primes

A small helper function is needed to produce the first 64 primes. This version
uses basic trial division, which is more than enough for the size of this task.
The function returns the primes in a NumPy array for easier calculations later.


In [3]:
def primes(n: int) -> np.ndarray:
    """
    Return the first n prime numbers.
    """
    if n <= 0:
        return np.array([], dtype=int)

    found = []
    candidate = 2

    while len(found) < n:
        is_prime = True

        for p in found:
            if p * p > candidate:
                break
            if candidate % p == 0:
                is_prime = False
                break

        if is_prime:
            found.append(candidate)

        candidate += 1

    return np.array(found, dtype=int)


# Quick check
primes(10)


array([ 2,  3,  5,  7, 11, 13, 17, 19, 23, 29])